<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/misc/data-explore-iris-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration - Iris Data Set

This example explores the [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris) [[csv](https://github.com/venky14/Machine-Learning-with-Iris-Dataset/blob/master/Iris.csv)] from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) usings [Pandas Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).  This kind of [EDA]() (Exploratory Data Analysis) is one of the key data science components of machine learning applications.

<figure><br>
  <center><img src="../images/iris.jpg" /></center>
</figure>

Some of the example code here is pulled from the book [Neural Network Projects with Python](https://www.oreilly.com/library/view/neural-network-projects/9781789138900/) by James Loy. The majority is from the Github project [Machine Learning with Iris Dataset](https://github.com/venky14/Machine-Learning-with-Iris-Dataset) by [Veky Rathod](https://github.com/venky14)

## Data Set Information:

This is perhaps the best known dataset to be found in the pattern recognition literature. Fisher's paper is a classic in the field and is referenced frequently to this day. (See [Duda & Hart](https://drive.google.com/open?id=1fq7usrZu7nmi0co5urq60h9P6w0WkDbY), for example.) 

- The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant. 
- One class is linearly separable from the other 2; the latter are NOT linearly separable from each other. 

- Predicted attribute: class of iris plant. 

- This is an exceedingly simple domain. 

## Attribute Information:

1. [sepal](https://en.wikipedia.org/wiki/Sepal) length in cm 
2. [sepal](https://en.wikipedia.org/wiki/Sepal) width in cm 
3. [petal](https://en.wikipedia.org/wiki/Petal) length in cm 
4. [petal](https://en.wikipedia.org/wiki/Petal) width in cm 
5. class: 
  - Iris Setosa 
  - Iris Versicolour 
  - Iris Virginica
  
 <figure>
  <img src="../images/iris-petal-sepal.jpg" align=”right”/>
</figure>


# And finally some Python code

**Usage NOTE!** Use `Shift+Enter` to step through this notebook, executing the code as you go.

In [1]:
#@title Welcome
import datetime
print(f"Welcome to exploring this notebook at {datetime.datetime.now()}! ")

Welcome to exploring this notebook at 2019-08-09 08:54:59.557713! 


## Abstract Base Class : Layer

The abstract class Layer, which all other layers will inherit from, handles simple properties which are an input, an output, and both a forward and backward methods.

In [0]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [0]:
import pandas as pd
URL= 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

In [0]:
colnames = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv(URL, names=colnames)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
class           150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB
